# SUMO_DNDC

A surrogate model to LandscapeDNDC for faster inference... 😎

The plan is to model a time-series via a LSTM. Trainging data is provided by LandscapeDNDC model runs.

<figure>
    <img src="../images/sketch.png" alt="Sketch" width="400"/>
    <figcaption>Fig. 1: A sketch of the model setup</figcaption>
</figure>

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# general imports


In [4]:
# local imports


sketch.png
